In [1]:
import pandas as pd
import re
import string
import time
import numpy as np
from sklearn.metrics import precision_recall_fscore_support as score

In [2]:
stopwords = open('nepali_stopwords.txt','r').read().splitlines()

# Display all the stop words
with open('nepali_stopwords.txt', 'r') as f:
    print(f.read().splitlines())

['अक्सर', 'अगाडि', 'अझै', 'अनुसार', 'अन्तर्गत', 'अन्य', 'अन्यत्र', 'अन्यथा', 'अब', 'अरू', 'अर्को', 'अर्थात', 'अर्थात्', 'अलग', 'आए', 'आजको', 'आठ', 'आत्म', 'आदि', 'आफू', 'आफूलाई', 'आफैलाई', 'आफ्नै', 'आफ्नो', 'आयो', 'उन', 'उनको', 'उनले', 'उप', 'उहाँलाई', 'एउटै', 'एक', 'एकदम', 'औं', 'कतै', 'कम से कम', 'कसरी', 'कसै', 'कसैले', 'कहाँबाट', 'कहिलेकाहीं', 'कहिल्यै', 'कहीं', 'का', 'कि', 'किन', 'किनभने', 'कुनै', 'कुरा', 'कृपया', 'के', 'केहि', 'केही', 'को', 'कोही', 'क्रमशः', 'गए', 'जना', 'एक', 'दुई ', 'तीन ', 'चार ', 'पाँच ', 'छ ', 'सात', 'आठ ', 'नौ', 'दश ', 'खोला ', 'सडक ', 'महिला', 'पुरुष ', 'करिब', 'बालबालिका ', 'बालक', 'बालिका ', 'पुरै ', 'अहिले ', 'पहिले ', 'घर', 'र ', 'एउटा', 'अझै ', 'यो', 'त्यो ', 'त्यहाँ ', 'यस', 'जहाँ ', 'जम्मा', 'उस्तै ', 'को', 'नं ', 'प्रदेश ', 'चिनियाँ ', 'प्रधानमन्त्री ', 'ओली ', 'विप्लव ', 'देउवा ', 'वर्ष ', 'लागि ', 'कहिले ', 'चौथो ', 'तेस्रो ', 'अक्सर ', 'अनुसार ', 'अन्तर्गत ', 'अन्य ', 'अब ', 'अरु ', 'जताततै', 'जब', 'जबकि', 'जसको', 'जसबाट', 'जसमा', 'जसलाई', 'जसले'

In [3]:
negWords = open('neg.txt','r').read().splitlines()

# Display all the Negative words
with open('neg.txt', 'r') as f:
    print(f.read().splitlines())

['मृत्यु', 'दुर्घटना', 'ठक्कर', 'डुबेर', 'डुब्नु', 'मूल्य बृद्धि', 'मिलेन', 'भएन ', 'असफल', 'घाइते', 'प्रतिबन्ध', 'बिरामी', 'मूल्य बढ्यो', 'आन्दोलित', 'दुर्व्यवहार', 'हत्या', 'मर्यो', 'मर्नु', 'रोकियो', 'आशंका', 'शव', 'चलखेल', 'अवैध', 'नेतृत्वविहीन', 'दबाब', 'नहुने', 'जथाभावी', 'नीतिविपरीत', 'विपरीत', 'मार', 'सकेनन्', 'नसक्नु', 'घाटा', 'पराजित', 'हार', 'मिचेर', 'हड्ताल', 'नसक्ने', 'कारबाही', 'घट्यो', 'छैन', 'दुर्गती', 'निर्जन', 'टुहुरो', 'आलोच', 'संकट', 'डुबे', 'निषेध', 'नकारात्मक', 'शोषण', 'विकृति', 'नकार्न', 'हराएर', 'युद्ध', 'डराउने', 'सामन्त', 'दुर्घट', 'बेरोजगारी', 'बलात्कृत', 'मृतक', 'उत्पीडित', 'स्वप्नदोष', 'प्रतिमूर्ति', 'शिकार', 'धकेलिँदै', 'चोट', 'द्वन्द्वत्तरकाल', 'समवेद', 'कडा', 'कठिनाई', 'बहिर्गमन', 'झुलाएर', 'विघटन', 'भयभित', 'बिटर', 'जन्जीर', 'अल्झिए', 'आक्रोश', 'अनसन', 'कमजोरी', 'छल', 'बेपत्ता', 'काट्ने', 'विरोधसभा', 'पिछडिए', 'भ्रम', 'निरंकुशतन्त्र', 'विरुद्ध', 'लुट्ने', 'स्वार्थ', 'क्यान्सर', 'भनसुन', 'आतंक', 'बर्बाद', 'दब्नु', 'घाइते', 'विकट', 'अपहरण', 'कठोर', 'काट्द

In [4]:
posWords = open('positive_words.txt','r').read().splitlines()

# Display all the stop words
with open('positive_words.txt', 'r') as f:
    print(f.read().splitlines())

['समृद्धि', 'विजयी', 'सम्मान ', 'बढ्यो', 'फाइदा ', 'सहयोग ', 'प्रगति', 'सेवा', 'अठोट', 'आशा', 'न्याय ', 'पहल', 'संस्कृति', 'उद्धार ', 'स्थापित ', 'निरन्तर ', 'खुसी ', 'सुख', 'सुखद ', 'विकास ', 'निर्माण', 'सहायता', 'रमणीय ', 'रमाउनु', 'अपेक्षा ', 'महत्वपूर्ण ', 'अध्ययन ', 'विश्वास ', 'माया', 'प्रेम ', 'हिंसाविरुद्ध', 'भावनात्मक', 'उपचार ', 'योगदान ', 'योग्य', 'गर्व ', 'रेल ', 'जहाज', 'पानीजहाज ', 'क्षमता ', 'स्वीकार्य ', 'सहकार्य ', 'सुरक्षित ', 'क्रान्ति ', 'सुधार', 'दक्ष ', 'मिलाएर', 'भयो ', 'बन्ने ', 'सन्तुलित ', 'मौका ', 'प्रकाशित ', 'जित ', 'बलियो ', 'योग ', 'दिवस', 'दशैँ ', 'तिहार ', 'चाड ', 'छठ ', 'ल्होसार ', 'मनाइयो ', 'मनाउने ', 'हातेमालो ', 'अनर ', 'स्तरीय ', 'उच्चस्तरीय ', 'बजेट ', 'पारित ', 'सम्पदा ', 'बलियो', 'अब्बल ', 'बिहे ', 'सेमिनार ', 'टुङ्गो ', 'सुझाव ', 'पदमार्ग ', 'आयोजना', 'सम्पादन ', 'सुरु ', 'सफा', 'स्वच्छ ', 'अग्रता ', 'पर्यटक ', 'ओली ', 'प्रधानमन्त्री ', 'साधना ', 'निर्यात ', 'सपना ']


In [5]:
data = pd.read_csv("headlines.txt",sep="#", header=None)
data.columns=["body_text","label"]  #Initally header Name will be 0 & 1 so replace those Name
data.head()

,body_text,label
0,डुंगा दुर्घटनामा ९४ जना बेपता,neg
1,खोलामा डुबेर बालिकाको मृत्यु,neg
2,गाडिको ठक्करबाट एकको मृत्यु,neg
3,सडक दुर्घटनामा महिलाको मृत्यु,neg
4,करीब नौसय मेगावाट क्षमताका आयोजनामा चिनियाँ लगानी,neg


In [6]:
# Check the labels Name
data["label"].unique()

array(['neg', 'pos'], dtype=object)

In [7]:
data.loc[data['label']=='neg']
# data[data.label == 'neg']

,body_text,label
0,डुंगा दुर्घटनामा ९४ जना बेपता,neg
1,खोलामा डुबेर बालिकाको मृत्यु,neg
2,गाडिको ठक्करबाट एकको मृत्यु,neg
3,सडक दुर्घटनामा महिलाको मृत्यु,neg
4,करीब नौसय मेगावाट क्षमताका आयोजनामा चिनियाँ लगानी,neg
5,'मूल्य बृद्धि थोरै घटाउने होइन बढाएको पुरै फिर...,neg
6,संघीयता सफल बनाउनु अहिलेको आवश्यकता-मुख्यमन्त्...,neg
7,घर फर्किने चाँजो अझै मिलेन,neg
8,हात्तीलाई साङ्लो मुक्त गर्ने अभियान असफल,neg
9,"कोशी व्यारेजमा ट्रक खस्दा दुईको मृत्यु, ७ घाइते",neg


In [8]:
data.loc[data['label']=="pos"]

,body_text,label
47,नेपालमा चिनियाँ लगानी गणनायोग्य : प्रधानमन्त्री,pos
48,चन्द्रका ढुंगे मूर्तिमा लिच्छिविकालीन झझल्को,pos
49,कलेज एम्बेसडरका प्रतिभा,pos
50,कस्मिक लभ मा अर्पण र दिया,pos
51,विश्व आन्दोलन १९६८ को याद,pos
52,चारदामसँगै सकियो सेलो महोत्सव,pos
53,गर्छिन् पुकार आमाअनशन बसी जोडले...,pos
54,चीनमाथि थप ट्यारिफ लगाउने ट्रम्पको चेतावनी,pos
55,अालंकारिक उच्चताकाे सामन्तवाद,pos
56,असुली प्रेरित बन्द,pos


In [9]:
def stemword(word):
    
    x = re.findall(r'^((.*?)(लाई|ले|लागि|बाट|देखि|को|की|का|मा|माथि|कै|हरु|हरू|मै|न्ने|सँग|सँगै|वटा))$', word)
    if x:
        y = re.findall(r'^((.*?)(लाई|ले|लागि|बाट|देखि|को|की|का|मा|माथि|कै|हरु|हरू|मै|न्ने|सँग|सँगै|वटा))$', x[0][1])
        if y:	
            return y[0][1]
        else:
            return x[0][1]
    return word

In [10]:
print(stemword("राष्ट्रियसभालेलाई तीनै तहलाई समन्वय गर्ने"))

राष्ट्रियसभालेलाई तीनै तहलाई समन्वय गर्ने


In [11]:
# Stemword Function remove the stemword form word
print(stemword("राष्ट्रियसभालेलाई"))

राष्ट्रियसभा


In [12]:
def nepali_tokenize(text):
    return text.split()

In [13]:
# nepali_tokenize function takes sentence as arguments

print(nepali_tokenize("राष्ट्रियसभाले तीनै तहलाई समन्वय गर्ने"))

['राष्ट्रियसभाले', 'तीनै', 'तहलाई', 'समन्वय', 'गर्ने']


In [14]:
# string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

nepali_num = re.compile(r'(०|१|२|३|४|५|६|७|८|९)+')

# Suppose text = 'मानवअधिकार आयोगको :-  १.स्वतन्त्रतामा अंकुश नलगाउने'
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation]) 
    #O/P #'मानवअधिकार आयोगको १स्वतन्त्रतामा अंकुश नलगाउने आए'
    
    tokens = nepali_tokenize(text)
    #O/P ['मानवअधिकार', 'आयोगको', '१स्वतन्त्रतामा', 'अंकुश', 'नलगाउने','आए']
    
    text_no_num = [token for token in tokens if  not nepali_num.match(token)]
    #O/P ['मानवअधिकार', 'आयोगको', 'अंकुश', 'नलगाउने','आए']
    
    text = [stemword(word) for word in text_no_num if word not in stopwords]
    #O/P ['मानवअधिकार', 'आयोग', 'अंकुश', 'नलगाउने']
    
    return text

In [17]:
def negCount(text):
    text = clean_text(text)
    count = sum([1 for word in text if word in negWords])
    return count

def posCount(text):
    text = clean_text(text)
    count = sum([1 for word in text if word in posWords])
    return count

In [18]:
data['negCount'] = data['body_text'].apply(lambda x: negCount(x))
data['posCount'] = data['body_text'].apply(lambda x : posCount(x))
data.head()

,body_text,label,negCount,posCount
0,डुंगा दुर्घटनामा ९४ जना बेपता,neg,1,0
1,खोलामा डुबेर बालिकाको मृत्यु,neg,2,0
2,गाडिको ठक्करबाट एकको मृत्यु,neg,2,0
3,सडक दुर्घटनामा महिलाको मृत्यु,neg,2,0
4,करीब नौसय मेगावाट क्षमताका आयोजनामा चिनियाँ लगानी,neg,0,1


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[['body_text','negCount','posCount']], data['label'], test_size=0.2)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect_fit = tfidf_vect.fit(X_train['body_text'])

tfidf_train = tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test = tfidf_vect_fit.transform(X_test['body_text'])
print(len(tfidf_vect.get_feature_names()))
print(tfidf_vect.get_feature_names())

In [ ]:
# len(tfidf_vect.vocabulary_)

In [ ]:
# here 3600 is data size and 1846 is 

X_train.shape,tfidf_train.shape

In [ ]:
X_train_vect = pd.concat([X_train[['negCount','posCount']].reset_index(drop=True), 
           pd.DataFrame(tfidf_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['negCount','posCount']].reset_index(drop=True), 
           pd.DataFrame(tfidf_test.toarray())], axis=1)


In [ ]:
X_train_vect

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_vect, y_train)
y_pred = clf.predict(X_test_vect)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred,y_test)

In [ ]:
precision, recall, fscore, train_support = score(y_test, y_pred, pos_label='neg', average='binary')
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3), round(recall, 3),
        round((y_pred == y_test).sum() / len(y_pred), 3)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

In [ ]:
# import pickle
# pickle.dump(tfidf_vect_fit, open('final_vector.sav', 'wb'))
# pickle.dump(model, open('final_gbc.sav', 'wb'))